In [ ]:
# -*- coding: utf-8 -*-
"""
Multiflux Theory v2.0 — Teste Variacional para N=384³
Configuração otimizada para memória reduzida (float32) e cálculos eficientes
"""

import numpy as np
from scipy.fft import fftn, ifftn, fftfreq
from sklearn.cluster import KMeans
import os
import matplotlib.pyplot as plt

# -----------------------
# Configuração Otimizada
# -----------------------
N = 384                    # 384³ = 56M pontos
L = 2 * np.pi
SEED = 42
K_CLUSTERS = 12
VOLUME_CUTOFF = 0.005
SAVE_DIR = f"results/multiflux_v2_N{N}"
os.makedirs(SAVE_DIR, exist_ok=True)

np.random.seed(SEED)

# Use float32 para reduzir a memória pela metade
dtype = np.float32

# -----------------------
# 1) Rede de wavenumbers (float32)
# -----------------------
kx = fftfreq(N, d=L/N).astype(dtype) * 2 * np.pi
KX, KY, KZ = np.meshgrid(kx, kx, kx, indexing='ij')
K2 = (KX**2 + KY**2 + KZ**2).astype(dtype) + 1e-30
K = np.sqrt(K2)

# -----------------------
# 2) Amplitudes + espectro Kolmogorov (float32)
# -----------------------
amp = (np.random.randn(3, N, N, N) + 1j * np.random.randn(3, N, N, N)).astype(np.complex64)

# Shell binning simplificado
k_flat = K.ravel()
k_nonzero = k_flat[k_flat > 0]
kmin, kmax = k_nonzero.min(), k_nonzero.max()
n_shells = int(np.ceil(np.sqrt(3) * (N//2)))
shell_edges = np.linspace(kmin, kmax, n_shells + 1)
k_shell_idx = np.digitize(K, shell_edges)

power_shell = np.zeros(n_shells + 2, dtype=dtype)
count_shell = np.zeros_like(power_shell, dtype=int)
amp_power = np.sum(np.abs(amp)**2, axis=0).astype(dtype)

for idx in range(1, n_shells + 1):
    mask = (k_shell_idx == idx)
    count = np.count_nonzero(mask)
    if count > 0:
        count_shell[idx] = count
        power_shell[idx] = amp_power[mask].sum()

shell_centers = 0.5 * (shell_edges[:-1] + shell_edges[1:])
desired = shell_centers**(-5/3 + 1e-10)  # evita zero
desired /= desired.sum() or 1
desired *= power_shell.sum()

for idx in range(1, n_shells + 1):
    mask = (k_shell_idx == idx)
    if np.any(mask):
        g = np.sqrt(desired[idx-1] / power_shell[idx]) if power_shell[idx] > 0 else 0
        amp[:, mask] *= g

# -----------------------
# 3) Projeção solenoidal
# -----------------------
k_dot_amp = KX * amp[0] + KY * amp[1] + KZ * amp[2]
amp[0] -= KX * (k_dot_amp / K2)
amp[1] -= KY * (k_dot_amp / K2)
amp[2] -= KZ * (k_dot_amp / K2)

# Hermitian symmetry (simplificado)
def enforce_hermitian(fh):
    fh = fh.copy()
    for i in range(1, N//2 + 1):
        ip = N - i
        fh[i] = np.conj(fh[ip])
        fh[ip] = np.conj(fh[i])
    return fh

for c in range(3):
    amp[c] = enforce_hermitian(amp[c])

# -----------------------
# 4) Campo físico (float32)
# -----------------------
u = np.real(ifftn(amp[0])).astype(dtype)
v = np.real(ifftn(amp[1])).astype(dtype)
w = np.real(ifftn(amp[2])).astype(dtype)

# -----------------------
# 5) Gradientes (apenas magnitude squared para dissipação)
# -----------------------
def grad_mag2(f):
    fhat = fftn(f)
    fx2 = np.real(ifftn(1j * KX * fhat))**2
    fy2 = np.real(ifftn(1j * KY * fhat))**2
    fz2 = np.real(ifftn(1j * KZ * fhat))**2
    return fx2 + fy2 + fz2

grad_u2 = grad_mag2(u) + grad_mag2(v) + grad_mag2(w)

# -----------------------
# 6) Invariantes simplificados para clustering
# -----------------------
vort_mag = np.sqrt(grad_mag2(w) - grad_mag2(v) + grad_mag2(u))  # Simplificado ou use Q aproximado

# -----------------------
# 7) Clustering (reduz memória com subsample se necessário)
# -----------------------
features = vort_mag.ravel()
kmeans = KMeans(n_clusters=K_CLUSTERS)
kmeans.fit(features.reshape(-1, 1))
labels = kmeans.labels_

# -----------------------
# 8) Teste Variacional (mesmo do anterior)
# -----------------------
KE = 0.5 * (u**2 + v**2 + w**2).mean()
DISS = grad_u2.mean()
L_effective = KE - DISS

# Salva os resultados
results = {
    "KE": KE,
    "DISS": DISS,
    "L_effective": L_effective,
    "N_eff": len(set(labels)),
}

# Salva resultados em arquivo
np.savetxt(os.path.join(SAVE_DIR, "multiflux_results.txt"), results)

print(f"Simulação finalizada com sucesso. Resultados salvos em: {SAVE_DIR}")
